In [1]:
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [2]:
data = np.loadtxt('grade.csv',delimiter=',')
row = data.shape[0]
col = data.shape[1]

label = data[:,col-3]
data = data[:,:col-3]

one = 0
two = 0
three = 0
four = 0
five = 0

for i in range(row):
    if label[i] <= 4:
        label[i] = 1
        one += 1
    elif label[i] <= 8:
        label[i] = 2
        two += 1
    elif label[i] <= 12:
        label[i] = 3
        three += 1
    elif label[i] <= 16:
        label[i] = 4
        four += 1
    elif label[i] <= 20:
        label[i] = 5
        five += 1
print(one,two,three,four,five, row)

2 109 156 109 19 395


In [3]:
train, test, train_label_1, test_label_1 = train_test_split(data, label, test_size=0.25, random_state = 58)

In [4]:
train = preprocessing.scale(train)
test = preprocessing.scale(test)
train_row = train.shape[0]
test_row = test.shape[0]

In [5]:
pca = PCA(n_components = 6)
train = pca.fit_transform(train)
test = pca.transform(test)

In [6]:
clf = svm.SVC(random_state = 58)
clf.fit(train, train_label_1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=58, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
c = 0
for i in range(train_row):
    if clf.predict([train[i,:]]) != train_label_1[i]:
        #print(i,clf.predict([train[i,:]]),train_label_1[i])
        c += 1
print(1-c/train_row)

c = 0
for i in range(test_row):
    if clf.predict([test[i,:]]) != test_label_1[i]:
        #print(i,clf.predict([test[i,:]]),test_label_1[i])
        c += 1
print(1-c/test_row)

0.6587837837837838
0.4040404040404041


In [8]:
param_grid = {'C':[0.1, 1, 10, 100], 'gamma':[1, 0.1, 0.01, 0.001]}
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose = 2)
grid.fit(train, train_label_1)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...........

C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................................ C=10, gamma=0.001, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................................ C=10, gamma=0.001, total=   0.0s
[CV] 

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.3s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [9]:
grid_predictions = grid.predict(test)
print(classification_report(test_label_1,grid_predictions))

             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00         1
        2.0       1.00      0.03      0.06        33
        3.0       0.35      0.81      0.49        36
        4.0       0.40      0.26      0.32        23
        5.0       0.00      0.00      0.00         6

avg / total       0.55      0.36      0.27        99



C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
